In [46]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Load datasets
customers_df = pd.read_csv("C:/Users/Ishan Das/OneDrive/Desktop/ZOPTAP ASSISMENT/Customers.csv")
products_df = pd.read_csv("C:/Users/Ishan Das/OneDrive/Desktop/ZOPTAP ASSISMENT/Products.csv")
transactions_df = pd.read_csv("C:/Users/Ishan Das/OneDrive/Desktop/ZOPTAP ASSISMENT/Transactions.csv")
# Convert 'SignupDate' and 'TransactionDate' to datetime
customers_df['SignupDate'] = pd.to_datetime(customers_df['SignupDate'])
transactions_df['TransactionDate'] = pd.to_datetime(transactions_df['TransactionDate'])

# Merge customer and transaction data
merged_df = pd.merge(transactions_df, customers_df, on='CustomerID')

# Merge with product data to get product details
merged_df = pd.merge(merged_df, products_df, on='ProductID')


In [48]:
# After merging the transactions with product data, check the column names
print(merged_df.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y'],
      dtype='object')


In [49]:
# Merge the transactions with product data (including 'Price' column)
merged_df = pd.merge(merged_df, products_df[['ProductID', 'Price']], on='ProductID', how='left')

# Check the column names again to ensure 'Price' is present
print(merged_df.columns)


Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Price_y', 'Price'],
      dtype='object')


In [50]:
# Check the data type of the 'Price' column
print(merged_df['Price'].dtype)

# If necessary, convert 'Price' to numeric
merged_df['Price'] = pd.to_numeric(merged_df['Price'], errors='coerce')


float64


In [51]:
# Monetary: Total spending per customer (sum of prices of products purchased)
monetary_df = merged_df.groupby('CustomerID')['Price'].sum().reset_index(name='Monetary')

# Check the result
print(monetary_df.head())


  CustomerID  Monetary
0      C0001   1391.67
1      C0002    835.68
2      C0003    782.83
3      C0004   1925.09
4      C0005    874.81


In [52]:
# Recency: How many days ago the customer made the last purchase
recency_df = merged_df.groupby('CustomerID')['TransactionDate'].max().reset_index()
recency_df['Recency'] = (datetime.now() - recency_df['TransactionDate']).dt.days

# Frequency: Total number of transactions per customer
frequency_df = merged_df.groupby('CustomerID').size().reset_index(name='Frequency')

# Monetary: Total spending per customer (sum of prices of products purchased)
monetary_df = merged_df.groupby('CustomerID')['Price'].sum().reset_index(name='Monetary')

# Product preferences: This could be the number of purchases per product category
category_df = merged_df.groupby(['CustomerID', 'Category']).size().unstack(fill_value=0)

# Merge all the features together
customer_features = recency_df.merge(frequency_df, on='CustomerID').merge(monetary_df, on='CustomerID')
customer_features = customer_features.merge(category_df, on='CustomerID')

# Merge with profile features (Region and SignupYear/Month)
customer_features = customer_features.merge(customers_df[['CustomerID', 'Region', 'SignupDate']], on='CustomerID')
customer_features['SignupYear'] = customer_features['SignupDate'].dt.year
customer_features['SignupMonth'] = customer_features['SignupDate'].dt.month

# Drop SignupDate column as it's no longer needed
customer_features = customer_features.drop(columns=['SignupDate'])

# Display the engineered features
print(customer_features.head())


  CustomerID     TransactionDate  Recency  Frequency  Monetary  Books  \
0      C0001 2024-11-02 17:04:16       86          5   1391.67      1   
1      C0002 2024-12-03 01:41:41       55          4    835.68      0   
2      C0003 2024-08-24 18:54:04      156          4    782.83      0   
3      C0004 2024-12-23 14:13:52       35          8   1925.09      3   
4      C0005 2024-11-04 00:30:22       84          3    874.81      0   

   Clothing  Electronics  Home Decor         Region  SignupYear  SignupMonth  
0         0            3           1  South America        2022            7  
1         2            0           2           Asia        2022            2  
2         1            1           2  South America        2024            3  
3         0            2           3  South America        2022           10  
4         0            2           1           Asia        2022            8  


In [54]:
# Select only the numeric columns for scaling (exclude CustomerID, Region, and any datetime columns)
numeric_features = customer_features.select_dtypes(include=['float64', 'int64'])

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(numeric_features)

# Display the scaled features (first 5 rows)
print(scaled_features[:5])


[[-0.26088317 -0.01145819  0.03332559 -0.3211125  -1.04160638  1.55087763
  -0.22104388]
 [-0.69916973 -0.46749414 -0.80691867 -1.22113205  0.77663634 -1.14846331
   0.67666495]
 [ 0.72879617 -0.46749414 -0.88678865 -1.22113205 -0.13248502 -0.248683
   0.67666495]
 [-0.98193525  1.35664965  0.83946076  1.47892659 -1.04160638  0.65109731
   1.57437379]
 [-0.28915972 -0.92353008 -0.74778315 -1.22113205 -1.04160638  0.65109731
  -0.22104388]]


In [55]:
# Calculate cosine similarity between customers based on the scaled features
cosine_sim = cosine_similarity(scaled_features)

# Convert the cosine similarity matrix to a DataFrame for easy manipulation
cosine_sim_df = pd.DataFrame(cosine_sim, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])


In [57]:
# Initialize a dictionary to store the top 3 lookalikes for each customer
lookalike_dict = {}

# Iterate over each customer to find the top 3 similar customers
for customer_id in customer_features['CustomerID'][:20]:  # First 20 customers (C0001 to C0020)
    # Get the similarity scores for the current customer
    similarity_scores = cosine_sim_df[customer_id]
    
    # Exclude the customer itself from the recommendations
    similarity_scores = similarity_scores.drop(customer_id)
    
    # Get the top 3 most similar customers (highest similarity scores)
    top_3_customers = similarity_scores.nlargest(3)
    
    # Store the results in the dictionary
    lookalike_dict[customer_id] = {
        'Lookalike1': top_3_customers.index[0], 
        'Score1': top_3_customers.values[0],
        'Lookalike2': top_3_customers.index[1],
        'Score2': top_3_customers.values[1],
        'Lookalike3': top_3_customers.index[2],
        'Score3': top_3_customers.values[2]
    }

# Convert the dictionary to a DataFrame for easy viewing
lookalike_df = pd.DataFrame.from_dict(lookalike_dict, orient='index')

# Save the result to a CSV file
lookalike_df.to_csv('Lookalike.csv')

# Display the first few rows of the resulting DataFrame
print(lookalike_df.head())


      Lookalike1    Score1 Lookalike2    Score2 Lookalike3    Score3
C0001      C0069  0.944409      C0146  0.784552      C0127  0.777754
C0002      C0133  0.966879      C0134  0.931345      C0159  0.903061
C0003      C0166  0.882103      C0128  0.854873      C0144  0.797472
C0004      C0017  0.915365      C0113  0.910606      C0194  0.893890
C0005      C0197  0.948760      C0140  0.885635      C0007  0.882779


In [58]:
import pandas as pd

# Assuming cosine_sim_df is your cosine similarity DataFrame
# and customer_features is your customer feature DataFrame.

# Initialize a dictionary to store the top 3 lookalikes for each customer
lookalike_dict = {}

# Iterate over each customer to find the top 3 similar customers
for customer_id in customer_features['CustomerID'][:20]:  # First 20 customers (C0001 to C0020)
    # Get the similarity scores for the current customer
    similarity_scores = cosine_sim_df[customer_id]
    
    # Exclude the customer itself from the recommendations
    similarity_scores = similarity_scores.drop(customer_id)
    
    # Get the top 3 most similar customers (highest similarity scores)
    top_3_customers = similarity_scores.nlargest(3)
    
    # Store the results in the dictionary with both customer IDs and their similarity scores
    lookalike_dict[customer_id] = {
        'Lookalike1': top_3_customers.index[0], 
        'Score1': top_3_customers.values[0],
        'Lookalike2': top_3_customers.index[1],
        'Score2': top_3_customers.values[1],
        'Lookalike3': top_3_customers.index[2],
        'Score3': top_3_customers.values[2]
    }

# Convert the dictionary to a DataFrame for easy viewing
lookalike_df = pd.DataFrame.from_dict(lookalike_dict, orient='index')

# Save the result to a CSV file
lookalike_df.to_csv('Lookalike.csv')

# Display the first few rows of the resulting DataFrame
print(lookalike_df.head())


      Lookalike1    Score1 Lookalike2    Score2 Lookalike3    Score3
C0001      C0069  0.944409      C0146  0.784552      C0127  0.777754
C0002      C0133  0.966879      C0134  0.931345      C0159  0.903061
C0003      C0166  0.882103      C0128  0.854873      C0144  0.797472
C0004      C0017  0.915365      C0113  0.910606      C0194  0.893890
C0005      C0197  0.948760      C0140  0.885635      C0007  0.882779
